# TP 2.2 Flambement et postflambement d'un poteau

Ceci est un exemple de solution du TP 2.2 (partie post-flambement). Quelques points à noter

1. Cette feuille de calcul ne fait pas appel à la fonction ``utils.plotbeam``. Une implémentation indépendante est proposée ci-après (voir ``deformed_beam``).
2. Le critère d'arrêt des itérations de Newton porte sur la norme de l'incrément, pas sur la norme du résidu (critère "d'absence de progrès").

*Auteur : Sébastien Brisard (2018)*

In [ ]:
import dolfin
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from dolfin import (cos, Constant, DirichletBC, dx, Expression, FiniteElement,
                    Function, FunctionSpace, interpolate,
                    LinearVariationalProblem, LinearVariationalSolver, norm,
                    solve, sin, TrialFunction, TestFunction, UnitIntervalMesh)

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
N = 100     # Number of elements
degree = 1  # Degree of finite elements
rtol = 1e-4
atol = 1e-6
max_iter = 50

In [ ]:
def deformed_beam(θ, degree=1):
    V = θ.function_space()
    test, trial = TestFunction(V), TrialFunction(V)
    bc = DirichletBC(V, 0., 'on_boundary && near(x[0], 0.)')
    a = trial.dx(0)*test*dx

    u = Function(V)
    solve(a == cos(θ)*test*dx, u, bc)

    v = Function(V)
    solve(a == sin(θ)*test*dx, v, bc)
    return u, v

In [ ]:
def initial_guess(κ0, V, degree=1):
    expr = Expression('{}+kappa0*x[0]'.format(np.pi/2), kappa0=κ0, degree=degree)
    return interpolate(expr, V)

In [ ]:
mesh = UnitIntervalMesh(N)

In [ ]:
element = FiniteElement('CG', mesh.ufl_cell(), degree=degree)
V = FunctionSpace(mesh, element)

In [ ]:
bcs = [DirichletBC(V, 0., "on_boundary && near(x[0], 0.)")]

In [ ]:
κ0 = Constant(0.1)
λ = Constant(1.0)

θ = Function(V)

Δθ = TrialFunction(V)
δθ = TestFunction(V)
a = (Δθ.dx(0)*δθ.dx(0)-λ*sin(θ)*Δθ*δθ)*dx
L = -((θ.dx(0)-κ0)*δθ.dx(0)+λ*cos(θ)*δθ)*dx

In [ ]:
Δθ = Function(V)
problem = LinearVariationalProblem(a, L, Δθ, bcs)
solver = LinearVariationalSolver(problem)

In [ ]:
κ0_vals = np.array([0.01, .1, .2, .4, .5,  1., 10.])
λ_vals = np.linspace(0., 5., num=21)
#λ_vals = np.array([2.5])
u_top = np.empty((κ0_vals.shape[0], λ_vals.shape[0]))
v_top = np.empty_like(u_top)

for i, κ0_val in enumerate(κ0_vals):
    κ0.assign(κ0_val)
    θ.assign(initial_guess(κ0_val, V))

    plt.figure()
    plt.title('κ₀ = {}'.format(κ0_val))
    plt.axis('equal')

    for j, λ_val in enumerate(λ_vals):
        λ.assign(λ_val)

        for k in range(max_iter):
            solver.solve()
            θ.assign(θ+Δθ)
            
            norm_Δθ = norm(Δθ, 'L2')
            norm_θ = norm(θ, 'L2')
            if norm_Δθ <= rtol*norm_θ+atol:
                break
        else:
            raise RuntimeError('could not converge {} {} {} {}'.format(κ0_val, λ_val, norm_Δθ, norm_θ))
            
        u, v = deformed_beam(θ)
        plt.plot(u.compute_vertex_values(),
                 v.compute_vertex_values(),
                 label='λ = {}'.format(λ_val))
        u_top[i, j] = u(1.0)
        v_top[i, j] = v(1.0)
    plt.legend()

In [ ]:
for i, κ0_val in enumerate(κ0_vals):
    plt.plot(λ_vals, np.abs(v_top[i]),
             label='κ₀ = {}'.format(κ0_val))
plt.legend()